# **Expectativa de Vida (OMS)**

O conjunto de dados relacionado à expectativa de vida e aos fatores de saúde de 193 países foi coletado do mesmo site do repositório de dados da OMS, e os dados econômicos correspondentes foram coletados do site das Nações Unidas.



Dentre todas as categorias de fatores relacionados à saúde, foram selecionados apenas os fatores críticos mais representativos. Observou-se que, nos últimos 15 anos, houve um grande desenvolvimento no setor de saúde, resultando na redução das taxas de mortalidade humana, especialmente nos países em desenvolvimento, em comparação com os últimos 30 anos. Portanto, neste projeto, consideramos dados do período de 2000 a 2015 para 193 países para análises posteriores.

# **Explicação das Colunas**


* Country - País                        
* Year - Idade                               
* Status - Status de país desenvolvido ou em desenvolvimento                            
* Life expectancy - Expectativa de vida por idade               
* Adult Mortality - Taxa de mortalidade adulta para ambos os sexos (probabilidade de morte entre 15 e 60 anos por 1000 habitantes)         
* infant deaths - Número de mortes infantis por 1000 habitantes
* Alcohol - Álcool, consumo per capita (15+) registado (em litros de álcool puro)
* percentage expenditure - Percentual de Gastos, despesas com saúde como porcentagem do Produto Interno Bruto per capita (%)
* Hepatitis B - Cobertura vacinal contra hepatite B (HepB) em crianças de 1 ano de idade (%)
* Measles - Sarampo, número de casos notificados por 1000 habitantes.
* BMI - IMC, Índice de Massa Corporal médio de toda a população
* under-five deaths - Número de mortes de crianças menores de cinco anos por 1000 habitantes
* Polio - Cobertura vacinal contra a poliomielite (Pol3) em crianças de 1 ano de idade (%)   
* Total expenditure - Despesas do governo geral com saúde como porcentagem das despesas totais do governo (%)
* Diphtheria - Cobertura da vacinação contra difteria, tétano e coqueluche (DTP3) em crianças de 1 ano de idade (%)
* HIV/AIDS - Mortes por 1.000 nascidos vivos por HIV/AIDS (0-4 anos)  
* GDP - PIB, Produto Interno Bruto per capita (em USD)  
* Population - População do País
* thinness  1-19 years - Magreza 10 - 19 anos, prevalência de magreza entre crianças e adolescentes de 10 a 19 anos (%)  
* thinness 5-9 years - Prevalência de magreza em crianças de 5 a 9 anos (%)     
* Income composition of resources - Índice de Desenvolvimento Humano em termos de composição da renda dos recursos (índice variando de 0 a 1)  
* Schooling - Escolarização, número de anos de escolaridade (anos)   




# **4. Processamento dos Dados**

Nesta etapa, é necessário que você:
* Faça a limpeza das colunas erradas;
* Faça o tratamento dos valores nulos da forma que achar válido;
* Trate as duplicatas;
* Transforme as colunas categóricas em numéricas;
* Faça o escalonamento de variáveis numéricas;

In [50]:
import pandas as pd
import numpy as np

df =pd.read_csv("./expectancy.csv")

# Define a opção para não usar notação científica para números de ponto flutuante
pd.set_option('display.float_format', lambda x: '%.0f' % x)


In [51]:

#Removendo o indice Unnamed e duplicidades
df_clean = df.drop(columns=['Unnamed: 0'])\
             .drop_duplicates()

#checando remoção de duplicidades
df_clean[df_clean.duplicated()]

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling


In [52]:

#Padronização de nomes de colunas snake_case (removendo / - spaces) substituindo por "_"
df_clean.columns = df_clean.columns.str.lower().str.replace(r'[ /-]+', '_', regex=True)

#df_clean.tail(5)

In [53]:

#Removendo sujeira no campo year
df_clean['year'] = df_clean['year'].astype(str).str.replace('O', '0')

#df_clean.tail(5)


In [54]:

#ajustando datatype da coluna year
df_clean['year'] = df_clean['year'].astype(int)

#df_clean.tail(20)

In [55]:
# Exemplo: Substituir NaN por zero
df_clean['population'] = df_clean['population'].fillna(0)
df_clean['population'] = df_clean['population'].astype(int)

#df_clean.tail(20)

In [56]:
#Padronizando casas decimais
df_clean['percentage_expenditure'] = df_clean['percentage_expenditure'].round()
df_clean['gdp'] = df_clean['gdp'].round()

#df_clean.tail(20)

In [57]:
#Trantando Nan por 0 em colunas numericas
for col in df_clean.columns:
    if df_clean[col].dtype in ['float64', 'int64']:
        df_clean[col] = df_clean[col].fillna(0)


#df_clean.info()
#df_clean.isnull().sum()

#df_clean.tail(20)

In [58]:
#Categoriazando a coluna status de forma manual

mapa_status = {
    'Developing': 0,
    'Developed': 1
}

df_clean['categoria_status'] = df_clean['status'].map(mapa_status)

#df_clean.tail(5)


In [62]:
from sklearn.preprocessing import StandardScaler

# Lista das colunas numéricas CONTÍNUAS que precisam ser escalonadas EXCETO a coluna categorica criada categoria_status e a coluna year
colunas_para_escalar = [
    'life_expectancy', 'adult_mortality', 'infant_deaths', 'alcohol', 
    'percentage_expenditure', 'hepatitis_b', 'measles', 'bmi', 
    'under_five_deaths', 'polio', 'total_expenditure', 'diphtheria', 
    'hiv_aids', 'gdp', 'population', 'thinness_1_19_years', 
    'thinness_5_9_years', 'income_composition_of_resources', 'schooling'
]

# 1. Instanciar o escalonador
scaler = StandardScaler()

# 2. Aplicar o FIT (aprender a média e o desvio padrão dos dados) e TRANSFORM (aplicar a transformação)
# Usamos .values para garantir que estamos alimentando o NumPy array subjacente
dados_escalonados = scaler.fit_transform(df_clean[colunas_para_escalar])

# 3. Criar um novo DataFrame ou substituir as colunas originais
df_clean[colunas_para_escalar] = dados_escalonados

print("Padronização (StandardScaler) aplicada.")
print("Média e Desvio Padrão das colunas escalonadas devem ser ~0 e ~1.")
print(df_clean[colunas_para_escalar].describe().loc[['mean', 'std']])
df_clean

Padronização (StandardScaler) aplicada.
Média e Desvio Padrão das colunas escalonadas devem ser ~0 e ~1.
      life_expectancy  adult_mortality  infant_deaths  alcohol  \
mean                0                0              0        0   
std                 1                1              1        1   

      percentage_expenditure  hepatitis_b  measles  bmi  under_five_deaths  \
mean                       0           -0       -0   -0                  0   
std                        1            1        1    1                  1   

      polio  total_expenditure  diphtheria  hiv_aids  gdp  population  \
mean      0                 -0          -0        -0    0           0   
std       1                  1           1         1    1           1   

      thinness_1_19_years  thinness_5_9_years  \
mean                   -0                  -0   
std                     1                   1   

      income_composition_of_resources  schooling  
mean                                0     

,country,year,status,life_expectancy,adult_mortality,infant_deaths,alcohol,percentage_expenditure,hepatitis_b,measles,...,total_expenditure,diphtheria,hiv_aids,gdp,population,thinness_1_19_years,thinness_5_9_years,income_composition_of_resources,schooling,categoria_status
0,Afghanistan,2015,Developing,-0,1,0,-1,-0,-0,-0,...,1,-1,-0,-0,0,3,3,-0,-0,0
1,Afghanistan,2014,Developing,-1,1,0,-1,-0,-0,-0,...,1,-1,-0,-0,-0,3,3,-0,-0,0
2,Afghanistan,2013,Developing,-1,1,0,-1,-0,-0,-0,...,1,-1,-0,-0,0,3,3,-0,-0,0
3,Afghanistan,2012,Developing,-1,1,0,-1,-0,0,0,...,1,-1,-0,-0,-0,3,3,-1,-0,0
4,Afghanistan,2011,Developing,-1,1,0,-1,-0,0,0,...,1,-1,-0,-0,-0,3,3,-1,-0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,Zimbabwe,2004,Developing,-2,4,-0,0,-0,0,-0,...,1,-1,6,-0,0,1,1,-1,-0,0
2934,Zimbabwe,2003,Developing,-2,4,-0,-0,-0,-2,-0,...,0,-1,7,-0,0,1,1,-1,-0,0
2935,Zimbabwe,2002,Developing,-2,-1,-0,0,-0,0,-0,...,0,-0,7,-0,-0,-1,-1,-1,-0,0
2936,Zimbabwe,2001,Developing,-2,4,-0,-1,-0,0,-0,...,0,-0,8,-0,0,-1,-1,-1,-0,0


In [60]:
# tratar nulos com 0

# life_expectancy
# adult_mortality
# alcohol
# hepatitis
# bmi
# total_expenditure tem sem info
# gpd tem sem info
# population tem sem info
# thinness
# income aqui deve variar de 0 a 1 mas existem sujeiras
# schooling


# alta cobertura de polio e difiteria + baixas taxas de magreza = alta expectativa de vida


# **5. Feature Engineer**

É aqui que sua imaginação precisa fluir. Crie de duas a três novas variáveis relacionando as variáveis já existentes e explique como elas poderiam agregar na capacidade do modelo de prever expectativa de vida.